In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [ ]:
#importamos la librería de openai para poder usar sus modelos del lenguaje
import openai
#importamos files de colab para poder cargar nuestros archivos de google colab
from google.colab import files

#introducimos nuestra clave de la api, para poder hacer uso de los servicios de Open AI
openai.api_key = 'your-api-key'

In [ ]:
#vamos a crear una función para cargar los contextos que ya habremos extraído manualmente de los documentos
#también podríamos automatizar esta parte hasta cierto punto y coger todos los documentos, buscar con expresiones regulares los enunciados
#en los que aparece el término y luego sacar una lista. Sin embargo, hemos preferido hacerlo de manera manual porque podemos darle
#parte del contexto en el que no aparezca necesariamente el término pero sí encontremos información que enriquezca el contexto.

def cargar_textos():
  contextos=[]
  continuar=True
  while continuar:
    print("A continuación, carga el archivo .txt que contiene uno de los contextos.\n")
    nube=files.upload()
    for clave in nube.keys():
      print(f"Archivo {clave} subido con éxito.\n")
      tx=nube[clave].decode('utf-8')
      contextos.append(tx)
    respuesta=input("¿Quieres cargar otro archivo? (s/n): ").strip().lower() #como files solo nos deja cargar un archivo cada vez, repetimos lo mismo hasta que el usuario no quiera cargar más archivos
    if respuesta != 's':
      continuar = False
  return contextos


In [ ]:
#ahora vamos a crear la función para extraer la información de los contextos proporcionados, esta será la primera llamada que haremos al modelo

def extractor_info(termino, contextos):
  prompt=f"Extrae la información más relevante de cada contexto sobre 'qué es {termino}' de los siguientes contextos, si no te ofrece información sobre qué es o sus características, no prestes atención al contexto:"
  for i, contexto in enumerate(contextos, 1):
    prompt+=f"[contexto {i}] '{contexto}' [fin contexto {i}], "
  respuesta = openai.Completion.create(engine="text-davinci-003", prompt=prompt, max_tokens=200, temperature=0.5)
  info=str(respuesta.choices[0].text.strip())
  return info


In [ ]:
#vamos a crear una función para la generación de las definiciones a partir de la información

def generacion_definicion(termino, resumen):
  prompt=f"Utilizando solo la información necesaria de la que te voy a proporcionar, debes elaborar una definición propia del término '{termino}', siguiendo la siguiente fórmula: Término = hiperónimo + características distintivas. Por ejemplo, 'Las eyectivas son [hiperónimo] [características distintivas]'. La información comienza aquí: {resumen}"
  respuesta=openai.Completion.create(engine="text-davinci-003", prompt=prompt, max_tokens=120, temperature=0.5)
  definicion=str(respuesta.choices[0].text.strip())
  return definicion


In [ ]:
def main():
  termino=input("Introduce el término a definir: ")
  contextos=cargar_textos()
  print("\n")
  print("Resumiendo contextos...\n")
  resumen=extractor_info(termino, contextos)
  print("Contextos resumidos con éxito.\n")
  print("Generando la definición...\n")
  definicion=generacion_definicion(termino, resumen)
  print("Definición generada con éxito.\n")
  print(f"Definicion de {termino}: {definicion}")


In [ ]:
if __name__=="__main__":
  main()